In [1]:
import os
from pathlib import Path
from collections import namedtuple
import threading, queue

FOLDER = Path(os.path.dirname(os.path.realpath("__file__"))) / 'data'
in_file = 'day11.txt'

In [2]:
Flash = namedtuple("flash", ('destination', 'name'), defaults=('flash',))
Tick = namedtuple("tick", ('destination', 'name'), defaults=('tick', ))

q = queue.Queue()

def task_runner():
    while True:
        event = q.get()
        event.destination.send(event)
        q.task_done()


class Listener:
    def __init__(self):
        self.events = {}
        
    def register_event(self, name, func):
        self.events[name] = func
        
    def send(self, event):
        name = event.name
        if name in self.events:
            self.events[name]()
    
            
class Octo(Listener):
    def __init__(self, energy, scheduler):
        super().__init__()
        self.energy = int(energy)
        self.flashcount = 0
        self.scheduler = scheduler
        self.flashing = False
        self.subscribers = []
        self.register_event('tick', self.tick)
        self.register_event('flash', self.receive_flash)
    
    def subscribe(self, other):
        self.subscribers.append(other)
    
    def increase_energy(self):
        if self.energy == 9:
            self.flash()
            self.energy = -1
        self.energy += 1
   
    def flash(self):
        if not self.flashing:
            self.flashcount += 1
            self.flashing = True
            for octo in self.subscribers:
                self.scheduler.put(Flash(octo))

    def receive_flash(self):
        if not self.flashing:
            self.increase_energy()

    def tick(self):
        self.flashing = False
        self.increase_energy()
        
    def __repr__(self):
        return f"{self.energy}"

In [5]:
with open(FOLDER / in_file) as f:            
    data = [[Octo(n, q) for n in line] for line in f.read().splitlines()]
    
    
def subscribe_neighbors(data):
    '''
    Mutually Subscribe to neighboring cells. 
    Returns flattened list of cephalopods
    '''
    all_octos = []
    shifts = [(-1, -1), (-1, 0), (0, -1), (-1, 1)]
    max_w = len(data[0])
    max_h = len(data) 
    for row, line in enumerate(data):
        for col, octo1 in enumerate(line):
            all_octos.append(octo1)
            for row_s, col_s in shifts:
                row_s += row
                col_s += col
                if 0 <= row_s < max_h and 0 <= col_s < max_w:
                    octo2 = data[row_s][col_s]
                    octo2.subscribe(octo1)
                    octo1.subscribe(octo2)
    return all_octos


def get_solutions(all_octos):
    threading.Thread(target=task_runner, daemon=True).start()    
    step = 0
    while True:
        step += 1
        for d in all_octos:
            q.put(Tick(d))
        q.join()
        if step == 100:
            solution1 = sum(o.flashcount for o in all_octos)
        if all(o.flashing for o in all_octos):
            solution2 = step
            break
            
    return solution1, solution2

In [6]:
all_octos = subscribe_neighbors(data)
get_solutions(all_octos)

(1601, 368)